In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np 
import glob
import os

In [2]:
tf.__version__

'2.3.0'

In [3]:
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()

In [4]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32') # 将数据转换为图像模式——单通道，然后转换数据类型为float32。

In [5]:
train_images = (train_images- 127.5) / 127.5  # 数据归一化

In [6]:
batch_size = 256
buffer_size = 60000

In [7]:
datasets = tf.data.Dataset.from_tensor_slices(train_images) # tensorflow原生方法存储数据

In [8]:
datasets = datasets.shuffle(buffer_size).batch(batch_size) # 打乱数据顺序，分批成簇

In [9]:
train_images.shape

(60000, 28, 28, 1)

In [10]:
def generator_model():
    model =  keras.Sequential()
    model.add(layers.Dense(256, input_shape=(100,), use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(28*28*1,  use_bias=False, activation='tanh'))
    model.add(layers.BatchNormalization())
    
    model.add(layers.Reshape((28, 28, 1)))
    
    return model

In [11]:
def discriminator_model():
    model = keras.Sequential()
    model.add(layers.Flatten())
    
    model.add(layers.Dense(512, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(256, use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Dense(1))
    
    return model

In [12]:
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

In [13]:
# 生成器损失计算。
def discriminator_loss(real_out, fake_out):
    real_loss = cross_entropy(tf.ones_like(real_out), real_out)
    fake_loss = cross_entropy(tf.zeros_like(fake_out), fake_out)
    return real_loss + fake_loss

In [14]:
# 判别器损失计算
def generator_loss(fake_out):
    fake_loss = cross_entropy(tf.ones_like(fake_out), fake_out)
    return fake_loss

In [16]:
generator_opt = tf.keras.optimizers.Adam(1e-4)
discriminator_opt = tf.keras.optimizers.Adam(1e-4)

In [17]:
epochs = 160
noise_dim = 100  # 每个噪声100维度
generate_image_num = 16  # 生成16个随机噪声
seed = tf.random.normal([generate_image_num, noise_dim])  # 16个随机噪声，用于可视化输出训练过程中的效果展示

In [18]:
generator = generator_model()
discriminator = discriminator_model()

In [19]:
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        real_out = discriminator(images, training=True)
        gen_image = generator(noise, training=True)
        fake_out = discriminator(gen_image, training=True)
        
        gen_loss = generator_loss(fake_out)
        disc_loss = discriminator_loss(real_out, fake_out)
    gradient_gen = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_disc = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    generator_opt.apply_gradients(zip(gradient_gen, generator.trainable_variables))
    discriminator_opt.apply_gradients(zip(gradient_disc, discriminator.trainable_variables))

In [20]:
def genetate_plot_images(gen_model, test_noise):
    pre_images = gen_model(test_noise, training=False)
    fig = plt.figure(figsize=(32, 128))
    for i in range(pre_images.shape[0]):
        plt.subplot(1, 16, i+1)
        plt.imshow((pre_images[i, :, :, 0] + 1)/2, cmap='gray')
        plt.axis('off')
    plt.show()

In [21]:
def train(dataset, epochs):
    for epoch in range(epochs):
        
        for image_batch in dataset:
            train_step(image_batch)
        if epoch % 20 == 0:
            print('---------------------------------------------------------------------------epoch:%s-----------------------------------------------------------------------------'%(epoch+1))
            genetate_plot_images(generator, seed)
    print('---------------------------------------------------------------------------epoch:%s-----------------------------------------------------------------------------'%(epoch+1))
    genetate_plot_images(generator, seed)

In [22]:
train(datasets, epochs)

KeyboardInterrupt: 